# Evaluate Multi Agent System Simulation

This notebook evaluates the accuracy and performance of the three-agent system of banks made in this repository with real world data.

## Evaluation with initial economic variables only

This section tests the multi-agent system pipeline with just the initialization of the economic variables. In this case, the agents have no prior information on their behavior in previous quarters. This evaluation thus relies solely on the LLM's training data to make good decisions and properly propagate changes in economic variables.

Hypothesis: the system will not match actual proceedings as the bank agents make the best decisions on all the information it knows (which is vastly more in some cases than some of the case studies' banks would have had access to) and the economic agent simply works off of these banks being representative of other similar banks.

In [ ]:
# Import GDP Growth Rate from FRED
import os

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()
# FRED API details
fred_api_url = "https://api.stlouisfed.org/fred/series/observations"
fred_api_key = os.getenv("FRED_API_KEY")
params = {
    "series_id": "A191RL1Q225SBEA",  # GDP growth rate series ID
    "api_key": fred_api_key,
    "file_type": "json",  # JSON format
    "frequency": "q",  # Quarterly frequency
    "start_date": "2010-01-01",  # Adjust as needed
    "end_date": "2024-01-01"  # Adjust as needed
}

# Fetch data from FRED API
response = requests.get(fred_api_url, params=params)
data = response.json()

# Parse JSON into a DataFrame
observations = data["observations"]
df = pd.DataFrame(observations)

# Clean and process the data
df["date"] = pd.to_datetime(df["date"])  # Convert date strings to datetime
df["value"] = pd.to_numeric(df["value"], errors="coerce")  # Convert GDP growth rate to numeric
df = df[["date", "value"]].rename(columns={"value": "Quarterly GDP Growth Rate"})

df["Quarterly GDP Growth Rate"] /= 100.0
print(df.head())